In [1]:
import pandas as pd
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()


i=0 ## batch counter

n=10000
sum_len = 20
sum_len_max=80

text_len = 200
text_len_max = 300


def count_ent(content):
    if count_ent.counter%1000 ==0:
        print('total processed:',count_ent.counter)
    count_ent.counter += 1
    temp_c = 0
    ents = nlp(content).ents
    for ent in ents:
        if ent.label_ in ['ORG', 'PRODUCT', 'FAC','WORK_OF_ART','MONEY']:
            temp_c = temp_c+1
    return temp_c
count_ent.counter=0

In [2]:
chunks = pd.read_json("downloads/corpus-webis-tldr-17.json", lines=True, chunksize = n)
for c in chunks:
    if i%n == 0:
        if i ==0:
            #df = c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']].reset_index(drop=True)
            df = c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
                       (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
                       ['content','summary','content_len','summary_len']].reset_index(drop=True)
        #else:
            ### save file
        #    filename = 'corpus_'+str(int(i/n))+'.csv'
        #    df.to_csv(r'output/corpus/'+filename)
            ### restart pd
        #    df = c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']].reset_index(drop=True)
    else:
        ### append new parts
        #df = df.append(c.loc[(c.content_len>text_len)&(c.summary_len>sum_len),['content','summary']], ignore_index=True)
        df = df.append(c.loc[(c.content_len>=text_len)&(c.content_len<=text_len_max)&
                       (c.summary_len>=sum_len)&(c.summary_len<=sum_len_max),
                             ['content','summary','content_len','summary_len']], ignore_index=True)
    if i % 50 ==0:
        print('currently processing: ', i)
    i=i+1
### save the last one
print('Done processing with length.')



currently processing:  0
currently processing:  50
currently processing:  100
currently processing:  150
currently processing:  200
currently processing:  250
currently processing:  300
currently processing:  350
Done processing with length.


In [15]:
lines = len(df)
lines

344121

In [17]:
df[0:3]

,content,summary,content_len,summary_len
0,I take a beta blocker for my heart condition t...,Butchered daughter to save the universe and wa...,219,27
1,Like I said I don't know and do not have the e...,I get you think that through some fancy hand-w...,212,39
2,"No, you're not alone. I didn't take one either...","The MSF, while a good base, isn't going to tra...",235,58


In [20]:
df.to_csv(r'output/corpus/len200_300.csv',index=False)

In [25]:
per_batch = int(lines/10)
for i in range(10):
    df2 = df[per_batch*i:per_batch*(i+1)]
    df2.to_csv(r'output/corpus/part'+str(i)+'.csv',index=False)

In [47]:
###### recognizing entities ##########
import time


for i in range(10):
    if i==0:
        continue
    start_t = time.time()
    df = pd.read_csv('output/corpus/part'+str(i)+'.csv')
    print('Processing batch: '+str(i))
    df['ent_count'] = df['content'].apply(lambda x : count_ent(x))
    df.to_csv(r'output/corpus/part'+str(i)+'.csv',index=False)
    end_t = time.time()
    print('spent time:',str(end_t-start_t))


Processing batch: 1
total processed: 0
total processed: 1000
total processed: 2000
total processed: 3000
total processed: 4000
total processed: 5000
total processed: 6000
total processed: 7000
total processed: 8000
total processed: 9000
total processed: 10000
total processed: 11000
total processed: 12000
total processed: 13000
total processed: 14000
total processed: 15000
total processed: 16000
total processed: 17000
total processed: 18000
total processed: 19000
total processed: 20000
total processed: 21000
total processed: 22000
total processed: 23000
total processed: 24000
total processed: 25000
total processed: 26000
total processed: 27000
total processed: 28000
total processed: 29000
total processed: 30000
total processed: 31000
total processed: 32000
total processed: 33000
total processed: 34000
spent time: 2278.623142004013
Processing batch: 2
total processed: 35000
total processed: 36000
total processed: 37000
total processed: 38000
total processed: 39000
total processed: 40000


In [29]:
df.to_csv(r'output/corpus/part0.csv',index=False)

In [34]:
df.loc[df.ent_count>2,['content','summary']].to_csv(r'output/corpus/part0_ent.csv',index=False)

In [ ]:
df = df.loc[df.ent_count>0,['content','summary','content_len','summary_len']].reset_index(drop=True)
print('Done filtering unrelated contents')

####### saving file ##############
#filename = 'corpus_last.csv'
df.to_csv(r'output/corpus/len200_300_filtered.csv',index=False) ### first round of filtering and processing
print('Done saving')

In [6]:
# read the split parts of processed data

df = pd.read_csv('output/corpus/part0.csv')
for i in range(1,10):
    df = df.append(pd.read_csv('output/corpus/part'+str(i)+'.csv'))

In [11]:
df = df.loc[df.ent_count>=2]

In [14]:
df[:10]

,content,summary,content_len,summary_len,ent_count
2,"No, you're not alone. I didn't take one either...","The MSF, while a good base, isn't going to tra...",235,58,4
3,I feel obliged to say that doing it properly m...,DMT is a drug unlike any other that can bring ...,228,78,4
6,"I've found that low-class mobs, Centurions and...",I don't know that I agree or disagree but I th...,210,25,7
10,Long-term CM player here.. started out playing...,I ramble my previous experiences with FM and I...,255,30,2
13,According to what they've said beginning nex...,Verizon isn't going to be left out in the cold...,264,21,10
15,"I'm not good at creating stories, and even wor...","Freya Black-Steel is a Nord woman, born in Mor...",236,44,10
16,"Bad writing. In all honesty though, Anakin is ...","Anakin was a weak will man child, the Sith wer...",207,38,4
18,I detest public employee unions - so I can't b...,This amendment does nothing to fix our $83 BIL...,265,32,2
19,"The Chinese system isn't exactly transparent, ...",2700 Delegates and representatives from all ov...,226,64,5
20,Ah Steel Archers I got you pinned down at la...,Basically started over with simpler graphics a...,275,24,4


In [12]:
df['content_len'].describe(include='all').apply(lambda x: format(x, 'f'))

count    134982.000000
mean        247.834245
std          28.972649
min         200.000000
25%         223.000000
50%         247.000000
75%         273.000000
max         300.000000
Name: content_len, dtype: object

In [13]:
df['summary_len'].describe(include='all').apply(lambda x: format(x, 'f'))

count    134982.000000
mean         35.548866
std          14.111320
min          20.000000
25%          24.000000
50%          31.000000
75%          43.000000
max          80.000000
Name: summary_len, dtype: object

In [18]:
df.to_csv('output/corpus/filtered.csv')

In [15]:
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.13.1


In [16]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [39]:
def clean_text(text, for_summary = False):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    if for_summary: ## remove puctuation for content
        text = re.sub(r'[_"\-%()|+&=*%:#@\[\]/]', ' ', text)
    else:
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\n', ' ', text)
    # Remove stopwords for contents, not summaries
    if not for_summary:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text) ## if more than 2 consecutive e or o, remove them
    text = re.sub(r"([^eoEO])\1\1+",r"\1",one) ## if more than 2 consecutive other characters, remove them
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text

In [40]:
df = pd.read_csv('output/corpus/filtered.csv')
df['content'] = df['content'].apply(lambda x: clean_text(x))
df['summary'] = df['summary'].apply(lambda x: clean_text(x, for_summary=True))

In [43]:
df = df.loc[:,['content','summary','content_len','summary_len','ent_count']]
df[:10]

,content,summary,content_len,summary_len,ent_count
0,alone take one either sit brc wife took instru...,"the msf, while a good base, is not going to tr...",235,58,4
1,feel obliged say properly makes huge differenc...,dmt is a drug unlike any other that can bring ...,228,78,4
2,found low class mobs centurions marauders nota...,i do not know that i agree or disagree but i t...,210,25,7
3,long term cm player started playing cm 92 93 a...,i ramble my previous experiences with fm and i...,255,30,2
4,according said beginning next year remove requ...,verizon is not going to be left out in the col...,264,21,10
5,good creating stories even worse writing freya...,"freya black steel is a nord woman, born in mor...",236,44,10
6,bad writing honesty though anakin portrayed se...,"anakin was a weak will man child, the sith wer...",207,38,4
7,detest public employee unions cannot believe s...,this amendment does nothing to fix our $83 bil...,265,32,2
8,chinese system exactly transparent several dec...,2700 delegates and representatives from all ov...,226,64,5
9,ah steel archers got pinned last took months f...,basically started over with simpler graphics a...,275,24,4


In [44]:
df.summary[1]

'dmt is a drug unlike any other that can bring you outside of your own consciousness. a drug i believe everyone who is considering should try correctly . edit forgot to mention that it usually lasts from 5 15 minutes when not combined with maois. i typically sprinkle it on a bowl of weed and hold a lighter far enough away to vaporize slightly combust the bud. imo if salvia is a freakish nightmare, dmt is a beautiful and lucid dream.'

In [46]:
df

,content,summary,content_len,summary_len,ent_count
0,alone take one either sit brc wife took instru...,"the msf, while a good base, is not going to tr...",235,58,4
1,feel obliged say properly makes huge differenc...,dmt is a drug unlike any other that can bring ...,228,78,4
2,found low class mobs centurions marauders nota...,i do not know that i agree or disagree but i t...,210,25,7
3,long term cm player started playing cm 92 93 a...,i ramble my previous experiences with fm and i...,255,30,2
4,according said beginning next year remove requ...,verizon is not going to be left out in the col...,264,21,10
5,good creating stories even worse writing freya...,"freya black steel is a nord woman, born in mor...",236,44,10
6,bad writing honesty though anakin portrayed se...,"anakin was a weak will man child, the sith wer...",207,38,4
7,detest public employee unions cannot believe s...,this amendment does nothing to fix our $83 bil...,265,32,2
8,chinese system exactly transparent several dec...,2700 delegates and representatives from all ov...,226,64,5
9,ah steel archers got pinned last took months f...,basically started over with simpler graphics a...,275,24,4


In [45]:
df.to_csv('output/corpus/processed_1.csv',index=False)

In [33]:
df.content[1]

'feel obliged say properly makes huge difference course variety ways smoke vaporize key rule keeping lungs inhaled without step probably break many tales extraordinary experiences drug say every time broken profound memorable experience many people report seeing humanoid figures conscious beings aliens reaching cannot say consistently true 10 15 time tried broken body experiences maybe 8 times witnessed wholly hallucinated experiences giant flowers made soft peach colored flesh bouncing animal collective flying stone cavern blinding speed color changing rapids battles eyes closed say unlike every drug may reach body experience allows hallucinate things ordinarily would psychedelics like shrooms lsd prominent adjective would use describe true dmt experience would dream like meditative nightmarish frightening never seen anyone bad time dmt'

In [42]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\betty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [45]:
df.to_csv(r'output/corpus/part0_ent.csv',index=False)